# Week 11 - Supervised Learning 

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

This week we will be applying some common supervised learning methods to classify the Iris dataset.
    
I have provided links to online tutorials for you learn how to use some basic scikit-learn classifiers by yourself. 
    
In the tutorial we will discuss how to assess the quality of models that you generate using the confusion matix and a number of metrics discussed in lectures.

</div>

## Classification Algorithms 

[Intro to Machine Learning](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)

In general, a learning problem considers a set of n samples of data and then tries to predict properties of unknown data. If each sample is more than a single number and, for instance, a multi-dimensional entry (aka multivariate data), it is said to have several attributes or features.

Learning problems fall into a few categories:

- **Supervised learning**, in which the data comes with additional attributes that we want to predict [Click here to go to the scikit-learn supervised learning page](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning).This problem can be either:

 - **Classification**: samples belong to two or more classes and we want to learn from already labeled data how to predict the class of unlabeled data. An example of a classification problem would be handwritten digit recognition, in which the aim is to assign each input vector to one of a finite number of discrete categories. Another way to think of classification is as a discrete (as opposed to continuous) form of supervised learning where one has a limited number of categories and for each of the n samples provided, one is to try to label them with the correct category or class.
 - **Regression**: if the desired output consists of one or more continuous variables, then the task is called regression. An example of a regression problem would be the prediction of the length of a salmon as a function of its age and weight.

- **Unsupervised learning**, in which the training data consists of a set of input vectors x without any corresponding target values. The goal in such problems may be to discover groups of similar examples within the data, where it is called clustering, or to determine the distribution of data within the input space, known as density estimation, or to project the data from a high-dimensional space down to two or three dimensions for the purpose of visualization [Click here to go to the Scikit-Learn unsupervised learning page](https://scikit-learn.org/stable/unsupervised_learning.html#unsupervised-learning).



## Packages

In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns

## Import the dataset
To learn about classifiers will will be making use of the **sklearn library**. This library has some awesome tutorials and example datasets. One such dataset is the **Iris dataset**. The Iris flower data set or Fisher's Iris data set is a multivariate data set introduced by the British statistician and biologist Ronald Fisher in his 1936 paper *The use of multiple measurements in taxonomic problems as an example of linear discriminant analysis*. 

The data set consists of 50 samples from each of **three species** of Iris (*Iris setosa, Iris virginica and Iris versicolor*). 

**Four features** were measured from each sample: the length and the width of the sepals and petals, in centimeters.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/tutorials/media/week11/irises.jpeg" width="500">

In [27]:
# Load in the dataset
from sklearn import datasets
iris = datasets.load_iris()
# print(iris.DESCR)

## Explore the data

In [ ]:
# Let's unpack the data and associated labels
features = iris.feature_names
target_names = iris.target_names
petal_data = iris.data
labels = iris.target

print('feature names:\n', features)
print('\ntarget names:\n', target_names)
print('\nFirst 10 rows of data:\n', petal_data[:10])
print('\ntarget labels:', labels)

## Working with Numpy arrays

For this tutorial our petal data is stored as a 2 dimensional Numpy array.<br>
Each record in our dataset comprises measurements of petal length and width, and sepel length and width. These values are stored in a numpy array.

Unlike pandas DataFrames, Numpy ndarrays are *not necessarily* tabular data (ie possessing rows and columns).<br>
They can have an arbitrary number of dimensions, defined by the user. 

Some examples of data we can store using Numpy ndarrays:
- 1 dimensional array: A vector of length 10; shape=(10,)
- 2 dimensional array: Tabular data with 10 observations of 4 features; shape=(10,4)
- 3 dimensional array: An RGB color image of 256x256 pixels; shape=(3,256,256) (as we need a separate matrix for each color channel).

Numpy ndarrays are an efficient way to work with large multidimentional datasets. Let's explore how we can interact with them.

> Note <br>
> For more info on working with arrays and all the cool things they can do check out the Numpy docs or the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html).

**Data format**


In [ ]:
# Run this cell to view how our data is stored. 
print(f'Dimensions: {petal_data.ndim}')
print(f'Shape: {petal_data.shape}')
print(f'Size: {petal_data.size}')

**Array indexing - access single elements**

In [ ]:
# Accessing the first observation in our dataset (ie a row)
print(petal_data[0])

# Lower dims in our array can be accessed by chaining index calls
print(petal_data[0][1])

# Or by comma separated indexing of each dim
print(petal_data[0, 2])

**Slicing multidimentional arrays**

Numpy ndarray slicing syntax follows the conventions seen in base python. i.e. uses [start:stop:step] notation.<br>
We can slice each dimension of the array simultaneously using commas to separate the slices

In [ ]:
# The first 5 observations
petal_data[:5]

In [ ]:
# The first 2 values from each of the first 5 observations as a sub-array
petal_data[:5,:2]

In [ ]:
# The second column for the first five observations.
# This should be identical to the second column in the output of the cell above. 
# Note that the output is a 1D array (vector) this time rather than a 2D array! 
petal_data[:5, 1]

## Visualising the data

In [ ]:
# We can perform a PCA to get a look at the data 
# perform PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(petal_data)
print(f'explained variance ratio (first two components): {str(pca.explained_variance_ratio_)}')

# plotting
plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2
for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_pca[labels == i, 0], X_pca[labels == i, 1], color=color, alpha=.8, lw=lw, label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')
plt.show()

# Classifying the data

**Using scikit-learn classifiers**

In this tutorial we will use Naive Bayes, KNN, SVM, and Decision Trees to build predictive models. <br>
We will use classifiers provided by scikit-learn (sklearn). <br>


> Note <br>
> If interested, a comparison of the sklearn models on the iris dataset can be seen [at this link.](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py) 

Each classifier is a python class. To use a classifier we need to do the following:
- initialise the class, supplying any parameters the underlying model may need. 
- train the model using the .fit() function (supplying training data)
- predict new observations using the .predict() function (supplying test data)

The basic syntax is below. 

<div style='font-size: 13pt'>

```python
clf = Classifier(parameters)
clf.fit(trainingData, trainingLabels)
pred = clf.predict(testData)
```
</div>


**Training data and testing data**

Machine learning is about learning some properties of a data set and then testing those properties against another data set. A common practice in machine learning is to evaluate an algorithm by splitting a data set into two. We call one of those sets the **training set**, on which we learn some properties; we call the other set the **testing set**, on which we test the learned properties.


**Splitting the data**

To properly evaluate our model, we need to split the data into training and test sets.
The 'petal_data' is our data, and the 'labels' are each datapoint's class. 
Remember we want to guess (classify) a new iris flower to its correct label (iris flower type) given its petal data.

train_test_split will need to be given the petal_data, and the labels so it can split the data and labels in the same manner.
look up the [train_test_split documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to find how to do this.

In [ ]:
from sklearn.model_selection import train_test_split

# Create training petal_data, training labels, test petal_data, test lables using train_test_split()
data_train, data_test, label_train, label_test = train_test_split(petal_data, labels, test_size=0.6, random_state=10)

print('\nPreview of training data:')
print(data_train[:10])
print('\nPreview of training labels:')
print(label_train[:10])
print('\nPreview of test data:')
print(data_test[:10])
print('\nPreview of test labels:')
print(label_test[:10])

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

### Naive Bayes

<b>Material:</b> Scikit-learn implementation of [Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html).
    
<b>Challenge:</b> Use the .fit() and .predict() functions on the initialised classifier to train the model and predict new observations. 
    
</div>



In [ ]:
# our classifier class
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

# fitting the model & predicting test data.
# assign the predicted labels to a variable named pred_NB.
### YOUR CODE HERE
raise NotImplementedError

print('\ntrue:', label_test)
print('\npred:', pred_NB )
print("\nNumber of mislabeled points out of a total %d points : %d" % (data_test.shape[0], (label_test != pred_NB).sum()))



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

### K Nearest Neighbours

<b>Material:</b> Sample usage of [Nearest Neighbors classification](https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py) plotting decision boundaries
    
<b>Challenge:</b> Use the .fit() and .predict() functions on the initialised classifier to train the model and predict new observations. 
    
</div>

In [ ]:
# our classifier class
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=15, weights='uniform')

# fitting the model & predicting test data.
# assign the predicted labels to a variable named pred_KNN.
### YOUR CODE HERE
raise NotImplementedError

print('\ntrue:', label_test)
print('\npred:', pred_KNN )
print("\nNumber of mislabeled points out of a total %d points : %d" % (data_test.shape[0], (label_test != pred_KNN).sum()))



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

### Support Vector Machine

<b>Material:</b> Comparison of different [linear SVM classifiers](https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html) on a 2D projection of the iris dataset. 

<b>Challenge:</b> Use the .fit() and .predict() functions on the initialised classifier to train the model and predict new observations. 

NOTE: The classifier below is set up to use basic linear SVM. <br>
Scikit-learn additionally provides the SVC() class which can accept a kernel function.<br>
Provided kernels include 'poly', 'rbf', and 'sigmoid' which can be handy when the data is not linearly seperable.

</div>


In [ ]:
# our classifier class
from sklearn.svm import LinearSVC
clf = LinearSVC()

# fitting the model & predicting test data.
# assign the predicted labels to a variable named pred_SVM.
### YOUR CODE HERE
raise NotImplementedError

print('\ntrue:', label_test)
print('\npred:', pred_SVM )
print("\nNumber of mislabeled points out of a total %d points : %d" % (data_test.shape[0], (label_test != pred_SVM).sum()))



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

### Decision Trees

<b>Material:</b> Usage of the scikit-learn [Decision Tree Classifier](https://scikit-learn.org/dev/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

<b>Challenge:</b> Use the .fit() and .predict() functions on the initialised classifier to train the model and predict new observations. 

</div>


In [ ]:
# our classifier class
from sklearn.tree import DecisionTreeClassifier 
clf = DecisionTreeClassifier()

# fitting the model & predicting test data.
# assign the predicted labels to a variable named pred_DT.
### YOUR CODE HERE
raise NotImplementedError

print('\ntrue:', label_test)
print('\npred:', pred_DT )
print("\nNumber of mislabeled points out of a total {} points : {}".format(data_test.shape[0], (label_test != pred_DT).sum()))

# visualising our trained decision tree
print('\nFeature names ---')
for i, field in enumerate(features):
    print(f'x[{i}]: {field}')
print('\nLabel names ---')
print(target_names)

from sklearn.tree import plot_tree 
fig, ax = plt.subplots(figsize=(8, 8))
plot_tree(clf, ax=ax, fontsize=10)
plt.show()


# Assessing Models

**Graph Confusion Matrix Generation**

By definition a confusion matrix $C$ is such that $C_ij$ is equal to the number of observations known to be in group $i$ but predicted to be in group $j$.

Which is fancy jargon for this guy: 

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/tutorials/media/week11/CM.png" width="400">

Which we can use to derive information about how well our classifiers performed

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/tutorials/media/week11/errorTypes.png" width="400">

For example, lets say we have used one of the classifiers listed above on our training data and now we will asses how well it did by testing it on a separate dataset with 10 samples which results in the following confusion matrix: 

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/tutorials/media/week11/CM2.png" width="400">


**Making the confusion matrix with Python**

In python, you can code this with the following: 

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = ["T", "T", "T", "T", "F", "F", "F", "F", "F", "F"]
y_pred = ["T", "T", "T", "F", "T", "T", "F", "F", "F", "F"]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel() 
# Note: the ravel() function flattens a multidimensional array into a single flat array. 
print(f'tn: {tn}\nfp: {fp}\nfn: {fn}\ntp: {tp}')

**Confusion matrix for multiple classes**

For our data, there are 3 classes. How do we calculate these statistics?<br>
We calculate each statistic for a single class. <br>
For example, we would find TP, TN, FP, FP for iris setova. Then we would do the same for iris virginica etc.

In [ ]:
y_true = [1, 0, 0, 0, 2, 2, 1, 0, 1, 0, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 1]
y_pred = [0, 0, 0, 1, 2, 1, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 1, 1, 1, 0, 1]

cmatrix = confusion_matrix(y_true, y_pred)
labels = ['setosa', 'versicolor', 'virginica']
df = pd.DataFrame(data=cmatrix, index=labels, columns=labels)

fig, ax = plt.subplots(dpi=90, figsize=(4,4))
sns.heatmap(df, linewidth=3, cmap='Greys', annot=True, annot_kws={'size': 18}, robust=True, vmin=-100, vmax=1000, fmt="d", ax=ax, square=False, cbar=False)
plt.xlabel('True Class', c='green', size=14)
plt.ylabel('Predicted Class', c='red', size=14)
plt.show()

**Confusion Matrices for our models**

Run the following cell to compare the confusion matrices for our 4 classifiers.

Which type of iris flower is the hardest to predict? <br>
Which other type of iris flower is this most commonly misclassified as? 

In [ ]:
labels = ['setosa', 'versicolor', 'virginica']
fig, axes = plt.subplots(nrows=2, ncols=2, dpi=100)

i = 0
for name, pred in zip(['naive bayes', 'KNN', 'SVM', 'Decision Tree'], [pred_NB, pred_KNN, pred_SVM, pred_DT]):
    nb_cm = confusion_matrix(label_test, pred)
    df = pd.DataFrame(data=nb_cm, index=labels, columns=labels)
    ax = axes[i//2, i%2]
    sns.heatmap(df, linewidth=3, cmap='Greys', robust=True, annot=True, annot_kws={'size': 12}, vmin=-100, vmax=1000, fmt="d", ax=ax, square=False, cbar=False)
    ax.set_title(name)
    if i == 0:
        ax.set_xlabel('True Class', c='green', size=10)
        ax.set_ylabel('Predicted Class', c='red', size=10)
    i += 1
plt.tight_layout()
plt.show()

## Model Metrics

In our PCA projection of the Iris dataset we saw some overlap between *I. virginica* and *I. versicolor*. We might expect our classifiers to struggle with these samples. 

In this section we will be assessing how the Naive Bayes model performed on classifying samples with the true label **versicolor**.

In [ ]:
# First lets generate and print the array representing the Naive Bayes confusion matrix:
nb_cm = confusion_matrix(label_test, pred_NB)
print(nb_cm)

# We can extract the values we will need for versicolor like so
tp = nb_cm[1,1]
fn = nb_cm[2,1] + nb_cm[0,1]
fp = nb_cm[1,2] + nb_cm[1,0]
tn = nb_cm[0,0] + nb_cm[0,2] + nb_cm[2,0] + nb_cm[2,2] 


**Accuracy**

<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**How often is the classifier correct?**
    
Accuracy is the number of correct predictions made by the model over all predictions made. 

Accuracy is calculated as:<br><br>


$$
ACC = \frac{TP + TN}{TP + TN + FP + FN}
$$


<br>

Given the above equation, what is the **accuracy** of our classifier?
    
</div>

In [ ]:
# Calculate and print Accuracy statistic using the formula above and pre-calculated tn, fp, fn, tp variables
#tn, fp, fn, tp 

### YOUR CODE HERE
raise NotImplementedError

print(f'Accuracy: {acc}')

**Precision**

<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**When it predicts the positive result, how often is it correct?**

Precision is the proportion of predicted positives that truly are positives

$$
PPV = \frac{TP}{TP + FP}
$$

Given the above equation, what is the precision of our classifier?
</div>

In [ ]:
# Calculate and print Precision statistic using the formula above and pre-calculated tn, fp, fn, tp variables

### YOUR CODE HERE
raise NotImplementedError

print(f'Precision: {precision}')

**Recall (True Positive Rate)**

<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**When it is actually the positive result, how often does it predict correctly?**
    
Recall is the proportion of actual positives that are correctly classified<br>

$$
TPR = \frac{TP}{TP + FN}
$$

Given the above equation, what is the recall of our classifier?
</div>

In [ ]:
# Calculate and print Recall statistic using the formula above and pre-calculated tn, fp, fn, tp variables

### YOUR CODE HERE
raise NotImplementedError

print(f'Recall: {recall}')

**False Positive Rate**
<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**When it predicts a positive result, how often is it incorrect?**

$FPR = \frac{FP}{FP + TN}$

Given the above equation, what is the FPR of our classifier?
</div>

In [ ]:
# Calculate and print FPR using the formula above and pre-calculated tn, fp, fn, tp variables

### YOUR CODE HERE
raise NotImplementedError

print(f'False positive rate: {FPR}')

**F1 score**
<div style="color: rgb(0,96,100); background: rgb(178,235,242); border: solid 1px rgb(77,208,225); padding: 10px;">

**This is just the harmonic mean of precision and recall**
    
F1-score will penalise large discrepancies between precision and recall

$$
F_1 =  \frac{2*TP}{2*TP + FP + FN}
$$

Given the above equation, what is the F1 score of our classifier? 
</div>

In [ ]:
# Calculate and print F1 using the formula above and pre-calculated tn, fp, fn, tp variables

### YOUR CODE HERE
raise NotImplementedError

print(f'F1 score: {f1}')

In [24]:
# Hint at some helpful functions 
from sklearn.metrics import classification_report